<h1> Лабороторна робота N1
<h3>Частина 1

<h4>Хід виконання роботи</h4>
<ul>
<li>Створити віртуальне середовище (venv) в якому будуть встановлені всі необхідні бібліотеки та налаштування для даної лабораторної роботи;</li>
<li>Для кожної з адміністративних одиниць України завантажити (urllib) тестові структуровані файли, що містять значення VHI-індексу. При зберіганні файлу, до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, реалізувати механізм запобігання повторного довантаження та колізії даних;</li>

<li>Зчитати завантажені текстові файли у pandas dataframe. Здійснити data cleaning: прибрати зайві стовпці, заповнити пропуски, видалити зайвий текст тощо. 
Додати стовпчики з назвою та індексом області </li>
<li>Реалізувати процедуру зміни індексів: в завантажених з NOAA даних області індексуються за англійською абеткою (Province 1 - Cherkasy), потрібно замінити індекси так, щоб області індексувались за українською абеткою (1 область - Вінницька). 
Реалізувати процедури для формування вибірок (https://www.geeksforgeeks.org/pandas/ways-to-filter-pandas-dataframe-by-column-values/)  наступного виду:
<ul><li>Ряд VHI для області за вказаний рік;</li>
<li>Ряд VHI за вказаний діапазон років для вказаних областей;</li>
<li>Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;</li>
<li>Інші вибірки на вимогу викладача.</li></ul></li>
Всі процедури необхідно реалізувати окремими функціями та викликати їх в окремих комірках Jupyter Notebook. Додати текст завдань перед кожним пунктом та читабельний вивід функцій. 
</ul>


Імпорт потрібних бібліотек

In [45]:
import os
import urllib.request
import numpy as np
from datetime import datetime
import pandas as pd


Створення  потрібних змін для загрузки даних

In [46]:
BASE_URL = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
YEARS = (1981, 2024)  # період
SAVE_DIR = "data"
os.makedirs(SAVE_DIR, exist_ok=True)
PROVINCES = {
    24: "Vinnytsia", 25: "Volyn", 5: "Dnipropetrovsk", 6: "Donetsk",
    27: "Zhytomyr", 23: "Zakarpattia", 26: "Zaporizhzhia", 7: "Ivano-Frankivsk",
    11: "Kyiv", 13: "Kirovohrad", 14: "Luhansk", 15: "Lviv",
    16: "Mykolaiv", 17: "Odesa", 18: "Poltava", 19: "Rivne",
    21: "Sumy", 22: "Ternopil", 8: "Kharkiv", 9: "Kherson",
    10: "Khmelnytskyi", 1: "Cherkasy", 3: "Chernivtsi",
    2: "Chernihiv", 4: "Crimea", 12: "Kyiv City", 20: "Sevastopol"
}


Для кожної з адміністративних одиниць України завантажити (urllib) тестові структуровані файли, що містять значення VHI-індексу. При зберіганні файлу, до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, реалізувати механізм запобігання повторного довантаження та колізії даних;
Для цього створюю функцію download_vhi.

In [47]:
# === 1. Завантаження файлу ===
def download_vhi(province_id, province_name):
    """Завантаження VHI-файлу для області"""
    params = f"?country=UKR&provinceID={province_id}&year1={YEARS[0]}&year2={YEARS[1]}&type=Mean"
    url = BASE_URL + params
    filename = f"VHI_{province_id}_{province_name}.txt"
    filepath = os.path.join(SAVE_DIR, filename)

    if os.path.exists(filepath):
        print(f"[i] Файл для {province_name} уже є, пропуск...")
        return filepath

    try:
        print(f"[↓] Завантаження {province_name}...")
        urllib.request.urlretrieve(url, filepath)
        print(f"[✓] Збережено: {filepath}")
        return filepath
    except Exception as e:
        print(f"[!] Помилка завантаження {province_name}: {e}")
        return None

Функція що робить data-clining та зчитування даних.

In [61]:
# === 2. Читання одного файлу ===
def read_vhi_file(filepath, province_id, province_name):
    import re
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()

        # Відкидаємо все до <pre>
        match = re.search(r"<pre>(.*)</pre>", text, re.DOTALL)
        if match:
            data_text = match.group(1)
        else:
            data_text = text

        # Розбиваємо на рядки
        lines = data_text.strip().splitlines()

        clean_lines = []
        for line in lines:
            line = line.strip().rstrip(",")  # видаляємо кінець рядка комою
            if not line:
                continue
            parts = line.split(",")
            if len(parts) >= 7:
                try:
                    year = int(parts[0])
                    clean_lines.append(parts[:7])
                except ValueError:
                    continue

        if not clean_lines:
            print(f"[!] Дані не знайдені у файлі {filepath}")
            return pd.DataFrame()

        df = pd.DataFrame(clean_lines, columns=["year","week","SMN","SMT","VCI","TCI","VHI"])

        df = df.astype({
            "year": int,
            "week": int,
            "SMN": float,
            "SMT": float,
            "VCI": float,
            "TCI": float,
            "VHI": float
        })
        df = df[df["VHI"] != -1.0].reset_index(drop=True)
        df['province_id'] = province_id
        df['province_name'] = province_name

        return df

    except Exception as e:
        print(f"[!] Помилка обробки {province_name}: {e}")
        return pd.DataFrame()

Функція що об`єднує все в один DataFrame.

In [49]:
# === 3. Об'єднання всіх областей ===
def load_all_vhi():
    all_dfs = []
    for pid, pname in PROVINCES.items():
        filepath = download_vhi(pid, pname)
        if filepath:
            df = read_vhi_file(filepath, pid, pname)
            if not df.empty:
                all_dfs.append(df)

    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    else:
        return pd.DataFrame()

Фуекція що реалізує процедуру зміни індексів: в завантажених з NOAA даних області індексуються за англійською абеткою (Province 1 - Cherkasy), воно замінює індекси так, щоб області індексувались за українською абеткою (1 область - Вінницька).

In [ ]:
def reindex_provinces_ukr(df):    
    # Зіставлення англійських назв з українською
    english_to_ukr = {
        "Vinnytsia": "Вінницька", "Volyn": "Волинська", "Dnipropetrovsk": "Дніпропетровська",
        "Donetsk": "Донецька", "Zhytomyr": "Житомирська", "Zakarpattia": "Закарпатська",
        "Zaporizhzhia": "Запорізька", "Ivano-Frankivsk": "Івано-Франківська", "Kyiv": "Київська",
        "Kirovohrad": "Кіровоградська", "Luhansk": "Луганська", "Lviv": "Львівська",
        "Mykolaiv": "Миколаївська", "Odesa": "Одеська", "Poltava": "Полтавська",
        "Rivne": "Рівненська", "Sumy": "Сумська", "Ternopil": "Тернопільська",
        "Kharkiv": "Харківська", "Kherson": "Херсонська", "Khmelnytskyi": "Хмельницька",
        "Cherkasy": "Черкаська", "Chernivtsi": "Чернівецька", "Chernihiv": "Чернігівська",
        "Crimea": "Крим", "Kyiv City": "м.Київ", "Sevastopol": "Севастополь"
    }

    # Українські назви за абеткою
    ukr_provinces_sorted = [
        "Вінницька", "Волинська", "Дніпропетровська", "Донецька", "Житомирська",
        "Закарпатська", "Запорізька", "Івано-Франківська", "Київська", "Кіровоградська",
        "Луганська", "Львівська", "Миколаївська", "Одеська", "Полтавська",
        "Рівненська", "Сумська", "Тернопільська", "Харківська", "Херсонська",
        "Хмельницька", "Черкаська", "Чернівецька", "Чернігівська", "Крим",
        "м.Київ", "Севастополь"
    ]
    ukr_index_map = {name: idx + 1 for idx, name in enumerate(ukr_provinces_sorted)}

    # Копіюємо DataFrame, щоб не змінювати оригінал
    df = df.copy()
    df['province_id'] = df['province_name'].map(english_to_ukr).map(ukr_index_map)
    df['province_name'] = df['province_name'].map(english_to_ukr)
    # Перетворюємо англійські імена на українські, потім присвоюємо ID по алфавіту
   
    #df['province_name'] = df['province_name'].map(english_to_ukr)
    # Перевірка: якщо залишились NaN
    missing = df[df['province_id'].isna()]['province_name'].unique()
    if len(missing) > 0:
        print("[!]Не знайдені назви для переіндексації:", missing)

    return df

Завантаження наших даних

In [62]:
# === Основний блок ===
df_all = load_all_vhi()



[↓] Завантаження Vinnytsia...
[✓] Збережено: data\VHI_24_Vinnytsia.txt
[↓] Завантаження Volyn...
[✓] Збережено: data\VHI_25_Volyn.txt
[↓] Завантаження Dnipropetrovsk...
[✓] Збережено: data\VHI_5_Dnipropetrovsk.txt
[↓] Завантаження Donetsk...
[✓] Збережено: data\VHI_6_Donetsk.txt
[↓] Завантаження Zhytomyr...
[✓] Збережено: data\VHI_27_Zhytomyr.txt
[↓] Завантаження Zakarpattia...
[✓] Збережено: data\VHI_23_Zakarpattia.txt
[↓] Завантаження Zaporizhzhia...
[✓] Збережено: data\VHI_26_Zaporizhzhia.txt
[↓] Завантаження Ivano-Frankivsk...
[✓] Збережено: data\VHI_7_Ivano-Frankivsk.txt
[↓] Завантаження Kyiv...
[✓] Збережено: data\VHI_11_Kyiv.txt
[↓] Завантаження Kirovohrad...
[✓] Збережено: data\VHI_13_Kirovohrad.txt
[↓] Завантаження Luhansk...
[✓] Збережено: data\VHI_14_Luhansk.txt
[↓] Завантаження Lviv...
[✓] Збережено: data\VHI_15_Lviv.txt
[↓] Завантаження Mykolaiv...
[✓] Збережено: data\VHI_16_Mykolaiv.txt
[↓] Завантаження Odesa...
[✓] Збережено: data\VHI_17_Odesa.txt
[↓] Завантаження Poltav

Зчитування даних

In [63]:
for pid, pname in PROVINCES.items():
    filepath = os.path.join(SAVE_DIR, f"VHI_{pid}_{pname}.txt")
    df = read_vhi_file(filepath, pid, pname)
    print(pid, pname, len(df))


24 Vinnytsia 2186
25 Volyn 2186
5 Dnipropetrovsk 2186
6 Donetsk 2186
27 Zhytomyr 2186
23 Zakarpattia 2186
26 Zaporizhzhia 2186
7 Ivano-Frankivsk 2186
11 Kyiv 2186
13 Kirovohrad 2186
14 Luhansk 2186
15 Lviv 2186
16 Mykolaiv 2186
17 Odesa 2186
18 Poltava 2186
19 Rivne 2186
21 Sumy 2186
22 Ternopil 2186
8 Kharkiv 2186
9 Kherson 2186
10 Khmelnytskyi 2186
1 Cherkasy 2186
3 Chernivtsi 2186
2 Chernihiv 2186
4 Crimea 2186
12 Kyiv City 2186
20 Sevastopol 2186


Завантаження у один великий DataFrame

In [64]:
if not df_all.empty:  
    df_all = reindex_provinces_ukr(df_all)
    df_all.to_csv("all_vhi_cleaned.csv", index=False)        
    print("[✓] Дані збережено у all_vhi_cleaned.csv")
    print(df_all.head())
    # якщо треба подивитись сортування:
    print(df_all.sort_values('province_id'))
else:
    print("[!] Дані не були завантажені")

[✓] Дані збережено у all_vhi_cleaned.csv
   year  week    SMN     SMT    VCI    TCI    VHI  province_id province_name
0  1982     1  0.068  263.59  63.47  28.34  45.90            1     Вінницька
1  1982     2  0.074  265.78  67.62  23.05  45.34            1     Вінницька
2  1982     3  0.076  267.19  69.37  20.40  44.88            1     Вінницька
3  1982     4  0.075  268.57  65.26  17.93  41.60            1     Вінницька
4  1982     5  0.072  269.24  58.58  20.00  39.29            1     Вінницька
       year  week    SMN     SMT    VCI    TCI    VHI  province_id  \
0      1982     1  0.068  263.59  63.47  28.34  45.90            1   
1      1982     2  0.074  265.78  67.62  23.05  45.34            1   
2      1982     3  0.076  267.19  69.37  20.40  44.88            1   
3      1982     4  0.075  268.57  65.26  17.93  41.60            1   
4      1982     5  0.072  269.24  58.58  20.00  39.29            1   
...     ...   ...    ...     ...    ...    ...    ...          ...   
59017  

Функція що виводить ряд VHI для області за вказаний рік;

In [57]:
def vhi_by_region_year(df, province_name, year):
    """
    Повертає ряд VHI для конкретної області за вказаний рік.
    
    df : pd.DataFrame
    province_name : str - назва області українською
    year : int - рік
    """
    filtered = df[(df['province_name'] == province_name) & (df['year'] == year)]
    return filtered[['year', 'week', 'VHI', 'province_name']]
# ряд VHI для Києва за 2011
print(vhi_by_region_year(df_all, "Київська", 2011).head())

       year  week    VHI province_name
19396  2011     1  42.93      Київська
19397  2011     2  39.83      Київська
19398  2011     3  38.09      Київська
19399  2011     4  38.22      Київська
19400  2011     5  38.62      Київська


Функція що виводить ряд VHI за вказаний діапазон років для вказаних областей;


In [58]:
def vhi_by_regions_years(df, provinces, year_start, year_end):
    """
    Повертає ряд VHI для кількох областей за вказаний діапазон років.
    
    provinces : list[str] - список областей українською
    year_start : int
    year_end : int
    """
    filtered = df[
        (df['province_name'].isin(provinces)) &
        (df['year'] >= year_start) &
        (df['year'] <= year_end)
    ]
    return filtered[['year', 'week', 'VHI', 'province_name']]

# ряд VHI для Києва та Львова за 2010-2012
print(vhi_by_regions_years(df_all, ["Київська","Львівська"], 2010, 2012).head())

       year  week    VHI province_name
19344  2010     1  47.84      Київська
19345  2010     2  47.23      Київська
19346  2010     3  49.70      Київська
19347  2010     4  52.06      Київська
19348  2010     5  52.79      Київська


Функція що робить пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [60]:
def vhi_stats(df, provinces, years):
    """
    Обчислює min, max, mean, median VHI для вказаних областей та років.
    
    provinces : list[str] - список областей
    years : list[int] - список років
    """
    filtered = df[
        (df['province_name'].isin(provinces)) &
        (df['year'].isin(years))
    ]
    
    stats = filtered.groupby('province_name')['VHI'].agg(['min','max','mean','median']).reset_index()
    return stats

# статистика для Києва та Львова за 2010-2012
print(vhi_stats(df_all, ["Київська","Черкаська","Вінницька"], [2010,2011,2012]))


  province_name    min    max       mean  median
0     Вінницька  29.28  68.10  45.910256  43.985
1      Київська  26.82  74.38  44.714103  43.820
2     Черкаська  28.04  72.75  44.589744  42.680
